In [10]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [12]:
movies_data =pd.read_csv('/content/movies.csv')

In [18]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [19]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
3798,3798,4000000,Comedy Drama,NaN,60422,NaN,en,Another Happy Day,A wedding at her parents' Annapolis estate hur...,3.238466,...,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"At this wedding, the F-word stands for Family",Another Happy Day,6.2,37.0,Ellen Barkin Ezra Miller Kate Bosworth Demi Mo...,"[{'name': 'Sam Levinson', 'gender': 0, 'depart...",Sam Levinson
3799,3799,4000000,Adventure Action Thriller Crime,NaN,81390,terror scotland kidnapping nudity wilderness,en,A Lonely Place to Die,A group of five mountaineers are hiking and cl...,8.307230,...,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Out here, there's nowhere to hide.",A Lonely Place to Die,6.2,157.0,Melissa George Ed Speleers Eamonn Walker Alec ...,"[{'name': 'Steve Daly', 'gender': 0, 'departme...",Julian Gilbey
3800,3800,0,Comedy Fantasy Science Fiction,NaN,10981,loser bullying leere independent film best friend,en,Nothing,The film tells the story of two good friends w...,3.408955,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Nothing,5.8,38.0,David Hewlett Andrew Miller Gordon Pinsent Mar...,"[{'name': 'Michael Andrews', 'gender': 2, 'dep...",Vincenzo Natali
3801,3801,0,Drama,NaN,225235,alcohol adultery river camping province,ru,Географ глобус пропил,Victor Sluzhkin signs on as a teacher of geogr...,1.876621,...,120.0,"[{""iso_639_1"": ""ru"", ""name"": ""P\u0443\u0441\u0...",Released,NaN,The Geographer Drank His Globe Away,7.3,23.0,Konstantin Khabenskiy Elena Lyadova Aleksandr ...,"[{'name': 'Valery Todorovsky', 'gender': 2, 'd...",Aleksandr Veledinskiy
3802,3802,0,Drama Comedy History Music,NaN,14902,usa c,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
movies_data.shape

(3803, 24)

In [21]:
# selecting the relevant feature
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [22]:
movies_data.isnull().sum()

,0
index,0
budget,0
genres,0
homepage,2382
id,0
keywords,226
original_language,1
original_title,1
overview,2
popularity,1


In [23]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [24]:
# combining all selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [25]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
3798    Comedy Drama  At this wedding, the F-word stan...
3799    Adventure Action Thriller Crime terror scotlan...
3800    Comedy Fantasy Science Fiction loser bullying ...
3801    Drama alcohol adultery river camping province ...
3802                  Drama Comedy History Music usa c   
Length: 3803, dtype: object


In [26]:
# converting it into numerical values
vectorizer  = TfidfVectorizer()

In [27]:
feature_vector = vectorizer.fit_transform(combined_features)

In [28]:
print(feature_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 101933 stored elements and shape (3803, 14271)>
  Coords	Values
  (0, 168)	0.07577282932028595
  (0, 231)	0.08632919377181453
  (0, 4302)	0.10802576820257555
  (0, 11170)	0.10320179465216496
  (0, 4445)	0.10320179465216496
  (0, 2984)	0.22966270902270833
  (0, 2470)	0.22650920234981672
  (0, 4789)	0.16480948984209962
  (0, 11824)	0.33546161530554675
  (0, 13661)	0.1232930946591323
  (0, 2600)	0.24656428256435192
  (0, 11732)	0.21411180371382496
  (0, 4026)	0.23704580996246924
  (0, 12557)	0.07036243677389624
  (0, 14021)	0.12649979551885573
  (0, 9174)	0.09074110662959094
  (0, 9425)	0.26949828881299615
  (0, 10963)	0.1497279083891937
  (0, 14029)	0.23314874891759038
  (0, 14250)	0.20672870277406408
  (0, 10938)	0.21411180371382496
  (0, 11558)	0.20357519610117245
  (0, 13732)	0.19804791484117856
  (0, 12012)	0.15282238422201547
  (0, 7186)	0.22966270902270833
  :	:
  (3800, 2951)	0.27294847194226385
  (3800, 9763)	0.2729484

In [29]:
 # getting the similarity scores using cosine similarity
 similarity = cosine_similarity(feature_vector)

In [30]:
print(similarity)

[[1.         0.06982297 0.03673202 ... 0.03461548 0.         0.        ]
 [0.06982297 1.         0.03134024 ... 0.01074155 0.         0.        ]
 [0.03673202 0.03134024 1.         ... 0.         0.         0.        ]
 ...
 [0.03461548 0.01074155 0.         ... 1.         0.         0.01699966]
 [0.         0.         0.         ... 0.         1.         0.01098429]
 [0.         0.         0.         ... 0.01699966 0.01098429 1.        ]]


In [31]:
similarity.shape


(3803, 3803)

In [78]:
# getting the movie name from the user
movie_name = input(' Enter your favourite movie name : ')


 Enter your favourite movie name : india


In [100]:
# creating a list with all the movies
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [102]:
# Ensure movie_name is a string
movie_name = str(movie_name)

# Ensure list_of_all_titles contains strings
list_of_all_titles = [str(title) for title in list_of_all_titles]

# Now call the function
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Spider-Man', 'Inside Man', 'Superman']


In [89]:
close_match = find_close_match[0]
print(close_match)

Bandidas


In [90]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

In [91]:
print(index_of_the_movie)

1463


In [92]:
# according to the index we will get the list of the movies
similarity_score =list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.004997904145854758)), (1, np.float64(0.004381865244420664)), (2, np.float64(0.012965533728989013)), (3, np.float64(0.022588806085603916)), (4, np.float64(0.0048606616833444015)), (5, np.float64(0.005178648262810045)), (6, np.float64(0.0)), (7, np.float64(0.0050781843208984176)), (8, np.float64(0.0)), (9, np.float64(0.05175848715511019)), (10, np.float64(0.0056158374325144846)), (11, np.float64(0.03237291464077905)), (12, np.float64(0.0051219188599290226)), (13, np.float64(0.04403239601724501)), (14, np.float64(0.004955680093607614)), (15, np.float64(0.04139484252933428)), (16, np.float64(0.004819009027875148)), (17, np.float64(0.11573584172031992)), (18, np.float64(0.008372927134292711)), (19, np.float64(0.004331153123420447)), (20, np.float64(0.04389781862380507)), (21, np.float64(0.004199809675551781)), (22, np.float64(0.0)), (23, np.float64(0.0)), (24, np.float64(0.004653038774792855)), (25, np.float64(0.0)), (26, np.float64(0.0052301800842857946)), (27, np.float64

In [93]:
len(similarity_score)

3803

In [94]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(1463, np.float64(1.0000000000000002)), (918, np.float64(0.19131232392264166)), (167, np.float64(0.18151259113764218)), (3106, np.float64(0.1779704521721577)), (2349, np.float64(0.17560140011377295)), (1073, np.float64(0.17018132752112103)), (1090, np.float64(0.15149146027093457)), (1868, np.float64(0.1510233008390074)), (131, np.float64(0.14963708610439908)), (34, np.float64(0.14795114901503637)), (3320, np.float64(0.13632981684333154)), (1765, np.float64(0.13448116833092288)), (1266, np.float64(0.13147503604681482)), (3349, np.float64(0.13019886350659932)), (2065, np.float64(0.1281734635862428)), (2037, np.float64(0.1271752518966279)), (3269, np.float64(0.11926600274011476)), (1010, np.float64(0.11782068321916833)), (17, np.float64(0.11573584172031992)), (1952, np.float64(0.11415675335539202)), (806, np.float64(0.11408599420418755)), (575, np.float64(0.11127643598179657)), (70, np.float64(0.11025549557626374)), (3165, np.float64(0.10940144993837568)), (2473, np.float64(0.10840724510

In [95]:
print("Movies suggested for you : \n")

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Bandidas
2 . Inside Man
3 . Sahara
4 . High Heels and Low Lifes
5 . Small Time Crooks
6 . The River Wild
7 . All the Pretty Horses
8 . Cradle 2 the Grave
9 . G-Force
10 . Monsters University
11 . Extract
12 . The Newton Boys
13 . Cirque du Freak: The Vampire's Assistant
14 . Desperado
15 . To Rome with Love
16 . Diary of a Wimpy Kid: Dog Days
17 . Woman on Top
18 . Panic Room
19 . Pirates of the Caribbean: On Stranger Tides
20 . The Counselor
21 . Captain Corelli's Mandolin
22 . Vanilla Sky
23 . Wild Wild West
24 . Set It Off
25 . Vicky Cristina Barcelona
26 . Blow
27 . Once Upon a Time in Mexico
28 . The Good Night
29 . Cellular


Embedded text


In [103]:
movie_name = input(' Enter your favourite movie name : ')
movie_name = str(movie_name)


list_of_all_titles = [str(title) for title in list_of_all_titles]


find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
similarity_score =list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print("Movies suggested for you : \n")

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : spider man
Movies suggested for you : 

1 . Spider-Man
2 . Spider-Man 3
3 . Spider-Man 2
4 . The Notebook
5 . Seabiscuit
6 . Clerks II
7 . The Ice Storm
8 . Oz: The Great and Powerful
9 . Horrible Bosses
10 . The Count of Monte Cristo
11 . In Good Company
12 . Finding Nemo
13 . Clear and Present Danger
14 . Drag Me to Hell
15 . Brothers
16 . The Good German
17 . The Queen
18 . Escape from L.A.
19 . Hearts in Atlantis
20 . Labor Day
21 . Malone
22 . Wimbledon
23 . Daybreakers
24 . The Purge: Election Year
25 . The Life Aquatic with Steve Zissou
26 . Cold Mountain
27 . Bullets Over Broadway
28 . Melancholia
29 . Out of the Furnace
